In [1]:
%cd /opt

/opt


In [2]:
%%capture
!tar xvf /kaggle/input/extract-prebuilt-kaldi-from-docker/kaldi.tar

In [3]:
%cd /tmp

/tmp


In [4]:
!git clone https://github.com/pytorch/fairseq/

Cloning into 'fairseq'...
remote: Enumerating objects: 28172, done.
remote: Counting objects: 100% (586/586), done.
remote: Compressing objects: 100% (337/337), done.
remote: Total 28172 (delta 308), reused 464 (delta 236), pack-reused 27586
Receiving objects: 100% (28172/28172), 11.87 MiB | 16.27 MiB/s, done.
Resolving deltas: 100% (21131/21131), done.


In [5]:
%%capture
!pip install phonemizer

In [6]:
%%capture
!pip install git+https://github.com/pytorch/fairseq/

In [7]:
%%capture
!apt-get -y install espeak

In [8]:
!git clone https://github.com/kpu/kenlm

Cloning into 'kenlm'...
remote: Enumerating objects: 13824, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 13824 (delta 76), reused 92 (delta 45), pack-reused 13687
Receiving objects: 100% (13824/13824), 5.49 MiB | 11.79 MiB/s, done.
Resolving deltas: 100% (7956/7956), done.


In [9]:
%%capture
!apt-get -y install libeigen3-dev liblzma-dev zlib1g-dev libbz2-dev

In [10]:
%%capture
%cd kenlm
!mkdir build
%cd build
!cmake ..
!make -j 4
%cd /tmp

In [11]:
import os
os.environ['PATH'] = f"{os.environ['PATH']}:/tmp/kenlm/build/bin/"
os.environ['FAIRSEQ_ROOT'] = '/tmp/fairseq'

In [12]:
!cat /kaggle/input/wav2vec-u-cv-swedish-audio/*.wrd | grep -v '^$' | sort| uniq > /kaggle/working/sentences.txt


In [13]:
%cd fairseq/examples/wav2vec/unsupervised

/tmp/fairseq/examples/wav2vec/unsupervised


In [14]:
%%capture
!apt-get -y install zsh

In [15]:
!mkdir /kaggle/working/preppedtext

In [16]:
%cd scripts

/tmp/fairseq/examples/wav2vec/unsupervised/scripts


The next part requires a FastText language id model; I don't know where the 187 language model comes from, but there is a model for 176 languages [here](https://fasttext.cc/docs/en/language-identification.html#content)

In [17]:
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin

--2021-05-30 15:22:08--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131266198 (125M) [application/octet-stream]
Saving to: ‘lid.176.bin’

lid.176.bin         100%[===================>] 125.18M  50.5MB/s    in 2.5s    

2021-05-30 15:22:11 (50.5 MB/s) - ‘lid.176.bin’ saved [131266198/131266198]



In [18]:
!cat normalize_and_filter_text.py|sed -e 's/187/176/' > tmp
!mv tmp normalize_and_filter_text.py

In [19]:
# Needed to see what's going wrong
os.environ['HYDRA_FULL_ERROR'] = '1'

In [20]:
import os
os.environ['LD_LIBRARY_PATH'] = '/opt/conda/lib:/opt/kaldi/tools/openfst-1.6.7/lib:/opt/kaldi/src/lib'

There are two lines with missing variables in `prepare_text.sh` - [pull request](https://github.com/pytorch/fairseq/pull/3569) - so replace the file.

While I'm replacing the file: most of the first part of the script is unneeded, as I already have a phonetic dictionary, so I'm use that instead.

With the calls of the `preprocess.py` script, make sure to check the threshold: there's a divide by zero if the threshold is set too high.

Config options for `kaldi_initializer.py`

- `in_labels`: a naming component, for the Kaldi lexicons/fsts (required)
- `wav2letter_lexicon`: path to wav2letter lexicon
- `out_labels`: a naming component, for the Kaldi lexicons/fsts: set to `in_label` if missing
- `kaldi_root`: path to Kaldi: `/opt/kaldi` for my kaggle image
- `fst_dir`: path where generated fsts will be saved
- `data_dir`: path to phones data
- `lm_arpa`: path to the lm in ARPA format
- `blank_symbol`: CTC blank symbol (`<s>` here)
- `silence_symbol`: Kaldi symbol for silence (`<SIL>` is set for two of the scripts)

A config file needs to exist for this, even though the options set in it seem to be ignored.

In [21]:
!mkdir /tmp/fairseq/examples/speech_recognition/kaldi/config/

In [22]:
%%writefile /tmp/fairseq/examples/speech_recognition/kaldi/config/config.yaml
kaldi_root: "/opt/kaldi"

Writing /tmp/fairseq/examples/speech_recognition/kaldi/config/config.yaml


In [23]:
%%writefile prepare_text.sh
#!/usr/bin/env zsh
# Copyright (c) Facebook, Inc. and its affiliates.
#
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.

lg=$1
text_path=$2
target_dir=$3

#ph_lg=${lg:l}
#if test "$lg" = 'fr'; then
#  ph_lg='fr-fr'
#elif test "$lg" = 'en'; then
#  ph_lg='en-us'
#elif test "$lg" = 'pt'; then
#  ph_lg='pt-br'
#fi
ph_lg="sv"

echo $lg
echo $ph_lg
echo $text_path
echo $target_dir

mkdir -p $target_dir
#python normalize_and_filter_text.py --lang $lg < $text_path | grep -v '\-\-\-' >! $target_dir/lm.upper.lid.txt
#python $FAIRSEQ_ROOT/fairseq_cli/preprocess.py --dataset-impl mmap --trainpref $target_dir/lm.upper.lid.txt --only-source --destdir $target_dir --thresholdsrc 2 --padding-factor 1 --dict-only
#cut -f1 -d' ' $target_dir/dict.txt | grep -v -x '[[:punct:]]*' | grep -Pv '\d\d\d\d\d+' >! $target_dir/words.txt
cp /kaggle/input/wav2vec-u-cv-swedish-audio/train.wrd $target_dir/lm.upper.lid.txt
cut -f1 -d' ' /kaggle/input/wav2vec-u-cv-swedish-audio/dict.train >! $target_dir/words.txt

#one=$(echo "1" | PHONEMIZER_ESPEAK_PATH=$(which espeak) phonemize -p ' ' -w '' -l $ph_lg --language-switch remove-flags)
#sed 's/$/ 1/' $target_dir/words.txt | PHONEMIZER_ESPEAK_PATH=$(which espeak) phonemize -o $target_dir/phones.txt -p ' ' -w '' -l $ph_lg -j 70 --language-switch remove-flags
cut -f2- -d' ' /kaggle/input/wav2vec-u-cv-swedish-audio/dict.train >! $target_dir/phones.txt

#echo "one is ${one}"

#sed -i "s/${one}$//" $target_dir/phones.txt
#paste $target_dir/words.txt $target_dir/phones.txt >! $target_dir/lexicon.lst
cp /kaggle/input/wav2vec-u-cv-swedish-audio/dict.train $target_dir/lexicon.lst

#python $FAIRSEQ_ROOT/fairseq_cli/preprocess.py --dataset-impl mmap --trainpref $target_dir/phones.txt --only-source --destdir $target_dir/phones --thresholdsrc 1000 --padding-factor 1 --dict-only
python $FAIRSEQ_ROOT/fairseq_cli/preprocess.py --dataset-impl mmap --trainpref $target_dir/phones.txt --only-source --destdir $target_dir/phones --thresholdsrc 2 --padding-factor 1 --dict-only

python filter_lexicon.py -d $target_dir/phones/dict.txt < $target_dir/lexicon.lst >! $target_dir/lexicon_filtered.lst
python phonemize_with_sil.py -s 0.25 --surround --lexicon $target_dir/lexicon_filtered.lst < $target_dir/lm.upper.lid.txt >! $target_dir/phones/lm.phones.filtered.txt
cp $target_dir/phones/dict.txt $target_dir/phones/dict.phn.txt
echo "<SIL> 0" >> $target_dir/phones/dict.phn.txt
python $FAIRSEQ_ROOT/fairseq_cli/preprocess.py --dataset-impl mmap --trainpref $target_dir/phones/lm.phones.filtered.txt --workers 70 --only-source --destdir $target_dir/phones --srcdict $target_dir/phones/dict.phn.txt

lmplz -o 4 < $target_dir/lm.upper.lid.txt --discount_fallback --prune 0 0 0 3 >! $target_dir/kenlm.wrd.o40003.arpa
build_binary $target_dir/kenlm.wrd.o40003.arpa $target_dir/kenlm.wrd.o40003.bin
lg=$lg python $FAIRSEQ_ROOT/examples/speech_recognition/kaldi/kaldi_initializer.py fst_dir=$target_dir/fst/phn_to_words_sil lm_arpa=$target_dir/kenlm.wrd.o40003.arpa wav2letter_lexicon=$target_dir/lexicon_filtered.lst data_dir=$target_dir/phones "blank_symbol='<SIL>'" "in_labels='phn'" "kaldi_root='/opt/kaldi'"
lg=$lg python $FAIRSEQ_ROOT/examples/speech_recognition/kaldi/kaldi_initializer.py fst_dir=$target_dir/fst/phn_to_words lm_arpa=$target_dir/kenlm.wrd.o40003.arpa wav2letter_lexicon=$target_dir/lexicon_filtered.lst data_dir=$target_dir/phones  "in_labels='phn'" "kaldi_root='/opt/kaldi'"

lmplz -o 4 < $target_dir/phones/lm.phones.filtered.txt --discount_fallback >! $target_dir/phones/lm.phones.filtered.04.arpa
build_binary -s $target_dir/phones/lm.phones.filtered.04.arpa $target_dir/phones/lm.phones.filtered.04.bin
lmplz -o 6 < $target_dir/phones/lm.phones.filtered.txt --discount_fallback >! $target_dir/phones/lm.phones.filtered.06.arpa
build_binary -s $target_dir/phones/lm.phones.filtered.06.arpa $target_dir/phones/lm.phones.filtered.06.bin

lg=$lg python $FAIRSEQ_ROOT/examples/speech_recognition/kaldi/kaldi_initializer.py fst_dir=$target_dir/fst/phn_to_phn_sil lm_arpa=$target_dir/phones/lm.phones.filtered.06.arpa data_dir=$target_dir/phones "blank_symbol='<SIL>'" "in_labels='phn'" "kaldi_root='/opt/kaldi'"

Overwriting prepare_text.sh


In [24]:
#!echo | g++ -E -Wp,-v -
!find /usr/include -name 'iostream*'

/usr/include/boost/iostreams
/usr/include/boost/iostreams/detail/iostream.hpp
/usr/include/boost/typeof/std/iostream.hpp
/usr/include/c++/7/iostream


In [25]:
%%writefile /tmp/fairseq/examples/speech_recognition/kaldi/kaldi_initializer.py
#!/usr/bin/env python3

# Copyright (c) Facebook, Inc. and its affiliates.
#
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.

from dataclasses import dataclass
import hydra
from hydra.core.config_store import ConfigStore
import logging
from omegaconf import MISSING, OmegaConf
import os
import os.path as osp
from pathlib import Path
import subprocess
from typing import Optional

from fairseq.data.dictionary import Dictionary
from fairseq.dataclass import FairseqDataclass

script_dir = Path(__file__).resolve().parent
config_path = script_dir / "config"


logger = logging.getLogger(__name__)


@dataclass
class KaldiInitializerConfig(FairseqDataclass):
    data_dir: str = MISSING
    fst_dir: Optional[str] = None
    in_labels: str = MISSING
    out_labels: Optional[str] = None
    wav2letter_lexicon: Optional[str] = None
    lm_arpa: str = MISSING
    kaldi_root: str = MISSING
    blank_symbol: str = "<s>"
    silence_symbol: Optional[str] = None


def create_units(fst_dir: Path, in_labels: str, vocab: Dictionary) -> Path:
    in_units_file = fst_dir / f"kaldi_dict.{in_labels}.txt"
    if not in_units_file.exists():

        logger.info(f"Creating {in_units_file}")

        with open(in_units_file, "w") as f:
            print("<eps> 0", file=f)
            i = 1
            for symb in vocab.symbols[vocab.nspecial :]:
                if not symb.startswith("madeupword"):
                    print(f"{symb} {i}", file=f)
                    i += 1
    return in_units_file


def create_lexicon(
    cfg: KaldiInitializerConfig,
    fst_dir: Path,
    unique_label: str,
    in_units_file: Path,
    out_words_file: Path,
) -> (Path, Path):

    disambig_in_units_file = fst_dir / f"kaldi_dict.{cfg.in_labels}_disambig.txt"
    lexicon_file = fst_dir / f"kaldi_lexicon.{unique_label}.txt"
    disambig_lexicon_file = fst_dir / f"kaldi_lexicon.{unique_label}_disambig.txt"
    if (
        not lexicon_file.exists()
        or not disambig_lexicon_file.exists()
        or not disambig_in_units_file.exists()
    ):
        logger.info(f"Creating {lexicon_file} (in units file: {in_units_file})")

        assert cfg.wav2letter_lexicon is not None or cfg.in_labels == cfg.out_labels

        if cfg.wav2letter_lexicon is not None:
            lm_words = set()
            with open(out_words_file, "r") as lm_dict_f:
                for line in lm_dict_f:
                    lm_words.add(line.split()[0])

            num_skipped = 0
            total = 0
            with open(cfg.wav2letter_lexicon, "r") as w2l_lex_f, open(
                lexicon_file, "w"
            ) as out_f:
                for line in w2l_lex_f:
                    items = line.rstrip().split("\t")
                    assert len(items) == 2, items
                    if items[0] in lm_words:
                        print(items[0], items[1], file=out_f)
                    else:
                        num_skipped += 1
                        logger.debug(
                            f"Skipping word {items[0]} as it was not found in LM"
                        )
                    total += 1
            if num_skipped > 0:
                logger.warning(
                    f"Skipped {num_skipped} out of {total} words as they were not found in LM"
                )
        else:
            with open(in_units_file, "r") as in_f, open(lexicon_file, "w") as out_f:
                for line in in_f:
                    symb = line.split()[0]
                    if symb != "<eps>" and symb != "<ctc_blank>" and symb != "<SIL>":
                        print(symb, symb, file=out_f)

        lex_disambig_path = (
            Path(cfg.kaldi_root) / "egs/wsj/s5/utils/add_lex_disambig.pl"
        )
        res = subprocess.run(
            [lex_disambig_path, lexicon_file, disambig_lexicon_file],
            check=True,
            capture_output=True,
        )
        ndisambig = int(res.stdout)
        disamib_path = Path(cfg.kaldi_root) / "egs/wsj/s5/utils/add_disambig.pl"
        res = subprocess.run(
            [disamib_path, "--include-zero", in_units_file, str(ndisambig)],
            check=True,
            capture_output=True,
        )
        with open(disambig_in_units_file, "wb") as f:
            f.write(res.stdout)

    return disambig_lexicon_file, disambig_in_units_file


def create_G(
    kaldi_root: Path, fst_dir: Path, lm_arpa: Path, arpa_base: str
) -> (Path, Path):

    out_words_file = fst_dir / f"kaldi_dict.{arpa_base}.txt"
    grammar_graph = fst_dir / f"G_{arpa_base}.fst"
    if not grammar_graph.exists() or not out_words_file.exists():
        logger.info(f"Creating {grammar_graph}")
        arpa2fst = kaldi_root / "src/lmbin/arpa2fst"
        subprocess.run(
            [
                arpa2fst,
                "--disambig-symbol=#0",
                f"--write-symbol-table={out_words_file}",
                lm_arpa,
                grammar_graph,
            ],
            check=True,
        )
    return grammar_graph, out_words_file


def create_L(
    kaldi_root: Path,
    fst_dir: Path,
    unique_label: str,
    lexicon_file: Path,
    in_units_file: Path,
    out_words_file: Path,
) -> Path:
    lexicon_graph = fst_dir / f"L.{unique_label}.fst"

    if not lexicon_graph.exists():
        logger.info(f"Creating {lexicon_graph} (in units: {in_units_file})")
        make_lex = kaldi_root / "egs/wsj/s5/utils/make_lexicon_fst.pl"
        fstcompile = kaldi_root / "tools/openfst-1.6.7/bin/fstcompile"
        fstaddselfloops = kaldi_root / "src/fstbin/fstaddselfloops"
        fstarcsort = kaldi_root / "tools/openfst-1.6.7/bin/fstarcsort"

        def write_disambig_symbol(file):
            with open(file, "r") as f:
                for line in f:
                    items = line.rstrip().split()
                    if items[0] == "#0":
                        out_path = str(file) + "_disamig"
                        with open(out_path, "w") as out_f:
                            print(items[1], file=out_f)
                            return out_path

            return None

        in_disambig_sym = write_disambig_symbol(in_units_file)
        assert in_disambig_sym is not None
        out_disambig_sym = write_disambig_symbol(out_words_file)
        assert out_disambig_sym is not None

        try:
            with open(lexicon_graph, "wb") as out_f:
                res = subprocess.run(
                    [make_lex, lexicon_file], capture_output=True, check=True
                )
                assert len(res.stderr) == 0, res.stderr.decode("utf-8")
                res = subprocess.run(
                    [
                        fstcompile,
                        f"--isymbols={in_units_file}",
                        f"--osymbols={out_words_file}",
                        "--keep_isymbols=false",
                        "--keep_osymbols=false",
                    ],
                    input=res.stdout,
                    capture_output=True,
                )
                assert len(res.stderr) == 0, res.stderr.decode("utf-8")
                res = subprocess.run(
                    [fstaddselfloops, in_disambig_sym, out_disambig_sym],
                    input=res.stdout,
                    capture_output=True,
                    check=True,
                )
                res = subprocess.run(
                    [fstarcsort, "--sort_type=olabel"],
                    input=res.stdout,
                    capture_output=True,
                    check=True,
                )
                out_f.write(res.stdout)
        except subprocess.CalledProcessError as e:
            logger.error(f"cmd: {e.cmd}, err: {e.stderr.decode('utf-8')}")
            os.remove(lexicon_graph)
            raise
        except AssertionError:
            os.remove(lexicon_graph)
            raise

    return lexicon_graph


def create_LG(
    kaldi_root: Path,
    fst_dir: Path,
    unique_label: str,
    lexicon_graph: Path,
    grammar_graph: Path,
) -> Path:
    lg_graph = fst_dir / f"LG.{unique_label}.fst"

    if not lg_graph.exists():
        logger.info(f"Creating {lg_graph}")

        fsttablecompose = kaldi_root / "src/fstbin/fsttablecompose"
        fstdeterminizestar = kaldi_root / "src/fstbin/fstdeterminizestar"
        fstminimizeencoded = kaldi_root / "src/fstbin/fstminimizeencoded"
        fstpushspecial = kaldi_root / "src/fstbin/fstpushspecial"
        fstarcsort = kaldi_root / "tools/openfst-1.6.7/bin/fstarcsort"

        try:
            with open(lg_graph, "wb") as out_f:
                res = subprocess.run(
                    [fsttablecompose, lexicon_graph, grammar_graph],
                    capture_output=True,
                    check=True,
                )
                res = subprocess.run(
                    [
                        fstdeterminizestar,
                        "--use-log=true",
                    ],
                    input=res.stdout,
                    capture_output=True,
                )
                res = subprocess.run(
                    [fstminimizeencoded],
                    input=res.stdout,
                    capture_output=True,
                    check=True,
                )
                res = subprocess.run(
                    [fstpushspecial],
                    input=res.stdout,
                    capture_output=True,
                    check=True,
                )
                res = subprocess.run(
                    [fstarcsort, "--sort_type=ilabel"],
                    input=res.stdout,
                    capture_output=True,
                    check=True,
                )
                out_f.write(res.stdout)
        except subprocess.CalledProcessError as e:
            logger.error(f"cmd: {e.cmd}, err: {e.stderr.decode('utf-8')}")
            os.remove(lg_graph)
            raise

    return lg_graph


def create_H(
    kaldi_root: Path,
    fst_dir: Path,
    disambig_out_units_file: Path,
    in_labels: str,
    vocab: Dictionary,
    blk_sym: str,
    silence_symbol: Optional[str],
) -> (Path, Path, Path):
    h_graph = (
        fst_dir / f"H.{in_labels}{'_' + silence_symbol if silence_symbol else ''}.fst"
    )
    h_out_units_file = fst_dir / f"kaldi_dict.h_out.{in_labels}.txt"
    disambig_in_units_file_int = Path(str(h_graph) + "isym_disambig.int")
    disambig_out_units_file_int = Path(str(disambig_out_units_file) + ".int")
    if (
        not h_graph.exists()
        or not h_out_units_file.exists()
        or not disambig_in_units_file_int.exists()
    ):
        logger.info(f"Creating {h_graph}")
        eps_sym = "<eps>"

        num_disambig = 0
        osymbols = []

        with open(disambig_out_units_file, "r") as f, open(
            disambig_out_units_file_int, "w"
        ) as out_f:
            for line in f:
                symb, id = line.rstrip().split()
                if line.startswith("#"):
                    num_disambig += 1
                    print(id, file=out_f)
                else:
                    if len(osymbols) == 0:
                        assert symb == eps_sym, symb
                    osymbols.append((symb, id))

        i_idx = 0
        isymbols = [(eps_sym, 0)]

        imap = {}

        for i, s in enumerate(vocab.symbols):
            i_idx += 1
            isymbols.append((s, i_idx))
            imap[s] = i_idx

        fst_str = []

        node_idx = 0
        root_node = node_idx

        special_symbols = [blk_sym]
        if silence_symbol is not None:
            special_symbols.append(silence_symbol)

        for ss in special_symbols:
            fst_str.append("{} {} {} {}".format(root_node, root_node, ss, eps_sym))

        for symbol, _ in osymbols:
            if symbol == eps_sym or symbol.startswith("#"):
                continue

            node_idx += 1
            # 1. from root to emitting state
            fst_str.append("{} {} {} {}".format(root_node, node_idx, symbol, symbol))
            # 2. from emitting state back to root
            fst_str.append("{} {} {} {}".format(node_idx, root_node, eps_sym, eps_sym))
            # 3. from emitting state to optional blank state
            pre_node = node_idx
            node_idx += 1
            for ss in special_symbols:
                fst_str.append("{} {} {} {}".format(pre_node, node_idx, ss, eps_sym))
            # 4. from blank state back to root
            fst_str.append("{} {} {} {}".format(node_idx, root_node, eps_sym, eps_sym))

        fst_str.append("{}".format(root_node))

        fst_str = "\n".join(fst_str)
        h_str = str(h_graph)
        isym_file = h_str + ".isym"

        with open(isym_file, "w") as f:
            for sym, id in isymbols:
                f.write("{} {}\n".format(sym, id))

        with open(h_out_units_file, "w") as f:
            for sym, id in osymbols:
                f.write("{} {}\n".format(sym, id))

        with open(disambig_in_units_file_int, "w") as f:
            disam_sym_id = len(isymbols)
            for _ in range(num_disambig):
                f.write("{}\n".format(disam_sym_id))
                disam_sym_id += 1

        fstcompile = kaldi_root / "tools/openfst-1.6.7/bin/fstcompile"
        fstaddselfloops = kaldi_root / "src/fstbin/fstaddselfloops"
        fstarcsort = kaldi_root / "tools/openfst-1.6.7/bin/fstarcsort"

        try:
            with open(h_graph, "wb") as out_f:
                res = subprocess.run(
                    [
                        fstcompile,
                        f"--isymbols={isym_file}",
                        f"--osymbols={h_out_units_file}",
                        "--keep_isymbols=false",
                        "--keep_osymbols=false",
                    ],
                    input=str.encode(fst_str),
                    capture_output=True,
                    check=True,
                )
                res = subprocess.run(
                    [
                        fstaddselfloops,
                        disambig_in_units_file_int,
                        disambig_out_units_file_int,
                    ],
                    input=res.stdout,
                    capture_output=True,
                    check=True,
                )
                res = subprocess.run(
                    [fstarcsort, "--sort_type=olabel"],
                    input=res.stdout,
                    capture_output=True,
                    check=True,
                )
                out_f.write(res.stdout)
        except subprocess.CalledProcessError as e:
            logger.error(f"cmd: {e.cmd}, err: {e.stderr.decode('utf-8')}")
            os.remove(h_graph)
            raise
    return h_graph, h_out_units_file, disambig_in_units_file_int


def create_HLGa(
    kaldi_root: Path,
    fst_dir: Path,
    unique_label: str,
    h_graph: Path,
    lg_graph: Path,
    disambig_in_words_file_int: Path,
) -> Path:
    hlga_graph = fst_dir / f"HLGa.{unique_label}.fst"

    if not hlga_graph.exists():
        logger.info(f"Creating {hlga_graph}")

        fsttablecompose = kaldi_root / "src/fstbin/fsttablecompose"
        fstdeterminizestar = kaldi_root / "src/fstbin/fstdeterminizestar"
        fstrmsymbols = kaldi_root / "src/fstbin/fstrmsymbols"
        fstrmepslocal = kaldi_root / "src/fstbin/fstrmepslocal"
        fstminimizeencoded = kaldi_root / "src/fstbin/fstminimizeencoded"

        try:
            with open(hlga_graph, "wb") as out_f:
                res = subprocess.run(
                    [
                        fsttablecompose,
                        h_graph,
                        lg_graph,
                    ],
                    capture_output=True,
                    check=True,
                )
                res = subprocess.run(
                    [fstdeterminizestar, "--use-log=true"],
                    input=res.stdout,
                    capture_output=True,
                    check=True,
                )
                res = subprocess.run(
                    [fstrmsymbols, disambig_in_words_file_int],
                    input=res.stdout,
                    capture_output=True,
                    check=True,
                )
                res = subprocess.run(
                    [fstrmepslocal],
                    input=res.stdout,
                    capture_output=True,
                    check=True,
                )
                res = subprocess.run(
                    [fstminimizeencoded],
                    input=res.stdout,
                    capture_output=True,
                    check=True,
                )
                out_f.write(res.stdout)
        except subprocess.CalledProcessError as e:
            logger.error(f"cmd: {e.cmd}, err: {e.stderr.decode('utf-8')}")
            os.remove(hlga_graph)
            raise

    return hlga_graph


def create_HLa(
    kaldi_root: Path,
    fst_dir: Path,
    unique_label: str,
    h_graph: Path,
    l_graph: Path,
    disambig_in_words_file_int: Path,
) -> Path:
    hla_graph = fst_dir / f"HLa.{unique_label}.fst"

    if not hla_graph.exists():
        logger.info(f"Creating {hla_graph}")

        fsttablecompose = kaldi_root / "src/fstbin/fsttablecompose"
        fstdeterminizestar = kaldi_root / "src/fstbin/fstdeterminizestar"
        fstrmsymbols = kaldi_root / "src/fstbin/fstrmsymbols"
        fstrmepslocal = kaldi_root / "src/fstbin/fstrmepslocal"
        fstminimizeencoded = kaldi_root / "src/fstbin/fstminimizeencoded"

        try:
            with open(hla_graph, "wb") as out_f:
                res = subprocess.run(
                    [
                        fsttablecompose,
                        h_graph,
                        l_graph,
                    ],
                    capture_output=True,
                    check=True,
                )
                res = subprocess.run(
                    [fstdeterminizestar, "--use-log=true"],
                    input=res.stdout,
                    capture_output=True,
                    check=True,
                )
                res = subprocess.run(
                    [fstrmsymbols, disambig_in_words_file_int],
                    input=res.stdout,
                    capture_output=True,
                    check=True,
                )
                res = subprocess.run(
                    [fstrmepslocal],
                    input=res.stdout,
                    capture_output=True,
                    check=True,
                )
                res = subprocess.run(
                    [fstminimizeencoded],
                    input=res.stdout,
                    capture_output=True,
                    check=True,
                )
                out_f.write(res.stdout)
        except subprocess.CalledProcessError as e:
            logger.error(f"cmd: {e.cmd}, err: {e.stderr.decode('utf-8')}")
            os.remove(hla_graph)
            raise

    return hla_graph


def create_HLG(
    kaldi_root: Path,
    fst_dir: Path,
    unique_label: str,
    hlga_graph: Path,
    prefix: str = "HLG",
) -> Path:
    hlg_graph = fst_dir / f"{prefix}.{unique_label}.fst"

    if not hlg_graph.exists():
        logger.info(f"Creating {hlg_graph}")

        add_self_loop = script_dir / "add-self-loop-simple"
        kaldi_src = kaldi_root / "src"
        kaldi_lib = kaldi_src / "lib"

        try:
            if not add_self_loop.exists():
                fst_include = kaldi_root / "tools/openfst-1.6.7/include"
                add_self_loop_src = script_dir / "add-self-loop-simple.cc"

                subprocess.run(
                    [
                        "c++",
                        "-std=c++11",
                        f"-I{kaldi_src}",
                        f"-I{fst_include}",
                        f"-L{kaldi_lib}",
                        add_self_loop_src,
                        "-lkaldi-base",
                        "-lkaldi-fstext",
                        "-o",
                        add_self_loop,
                    ],
                    check=True,
                )

            my_env = os.environ.copy()
            my_env["LD_LIBRARY_PATH"] = f"{kaldi_lib}:{my_env['LD_LIBRARY_PATH']}"

            subprocess.run(
                [
                    add_self_loop,
                    hlga_graph,
                    hlg_graph,
                ],
                check=True,
                capture_output=True,
                env=my_env,
            )
        except subprocess.CalledProcessError as e:
            logger.error(f"cmd: {e.cmd}, err: {e.stderr.decode('utf-8')}")
            raise

    return hlg_graph


def initalize_kaldi(cfg: KaldiInitializerConfig) -> Path:
    if cfg.fst_dir is None:
        cfg.fst_dir = osp.join(cfg.data_dir, "kaldi")
    if cfg.out_labels is None:
        cfg.out_labels = cfg.in_labels

    kaldi_root = Path(cfg.kaldi_root)
    data_dir = Path(cfg.data_dir)
    fst_dir = Path(cfg.fst_dir)
    fst_dir.mkdir(parents=True, exist_ok=True)

    arpa_base = osp.splitext(osp.basename(cfg.lm_arpa))[0]
    unique_label = f"{cfg.in_labels}.{arpa_base}"

    with open(data_dir / f"dict.{cfg.in_labels}.txt", "r") as f:
        vocab = Dictionary.load(f)

    in_units_file = create_units(fst_dir, cfg.in_labels, vocab)

    grammar_graph, out_words_file = create_G(
        kaldi_root, fst_dir, Path(cfg.lm_arpa), arpa_base
    )

    disambig_lexicon_file, disambig_L_in_units_file = create_lexicon(
        cfg, fst_dir, unique_label, in_units_file, out_words_file
    )

    h_graph, h_out_units_file, disambig_in_units_file_int = create_H(
        kaldi_root,
        fst_dir,
        disambig_L_in_units_file,
        cfg.in_labels,
        vocab,
        cfg.blank_symbol,
        cfg.silence_symbol,
    )
    lexicon_graph = create_L(
        kaldi_root,
        fst_dir,
        unique_label,
        disambig_lexicon_file,
        disambig_L_in_units_file,
        out_words_file,
    )
    lg_graph = create_LG(
        kaldi_root, fst_dir, unique_label, lexicon_graph, grammar_graph
    )
    hlga_graph = create_HLGa(
        kaldi_root, fst_dir, unique_label, h_graph, lg_graph, disambig_in_units_file_int
    )
    hlg_graph = create_HLG(kaldi_root, fst_dir, unique_label, hlga_graph)

    # for debugging
    # hla_graph = create_HLa(kaldi_root, fst_dir, unique_label, h_graph, lexicon_graph, disambig_in_units_file_int)
    # hl_graph = create_HLG(kaldi_root, fst_dir, unique_label, hla_graph, prefix="HL_looped")
    # create_HLG(kaldi_root, fst_dir, "phnc", h_graph, prefix="H_looped")

    return hlg_graph


@hydra.main(config_path=config_path, config_name="kaldi_initializer")
def cli_main(cfg: KaldiInitializerConfig) -> None:
    container = OmegaConf.to_container(cfg, resolve=True, enum_to_str=True)
    cfg = OmegaConf.create(container)
    OmegaConf.set_struct(cfg, True)
    initalize_kaldi(cfg)


if __name__ == "__main__":

    logging.root.setLevel(logging.INFO)
    logging.basicConfig(level=logging.INFO)

    try:
        from hydra._internal.utils import (
            get_args,
        )  # pylint: disable=import-outside-toplevel

        cfg_name = get_args().config_name or "kaldi_initializer"
    except ImportError:
        logger.warning("Failed to get config name from hydra args")
        cfg_name = "kaldi_initializer"

    cs = ConfigStore.instance()
    cs.store(name=cfg_name, node=KaldiInitializerConfig)

    cli_main()

Overwriting /tmp/fairseq/examples/speech_recognition/kaldi/kaldi_initializer.py


In [26]:
!zsh prepare_text.sh sv /kaggle/working/sentences.txt /kaggle/working/preppedtext

sv
sv
/kaggle/working/sentences.txt
/kaggle/working/preppedtext
=== 1/5 Counting and sorting n-grams ===
Reading /kaggle/working/preppedtext/lm.upper.lid.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 14359 types 3160
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:37920 2:2571431424 3:4821433856 4:7714294272
Statistics:
1 3160 D1=0.722623 D2=1.14413 D3+=1.45956
2 10285 D1=0.848104 D2=1.2466 D3+=1.46191
3 12632 D1=0.943362 D2=1.24166 D3+=1.32723
4 19/11699 D1=0.970399 D2=1.4843 D3+=2.12351
Memory estimate for binary LM:
type     kB
probing 617 assuming -p 1.5
probing 764 assuming -r models -p 1.5
trie    309 without quantization
trie    182 assuming -q 8 -b 8 quantization 
trie    293 assuming -a 22 array pointer compression
trie    166 assuming -a 22 -q 8 -b 8 array pointer compression and quant